In [8]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )
pts = [Point.random() for n in range(5)]

# Elements of a Function


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.P03.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D06.jpg" style="width: 600px; display: inline;">

## Procedures

In [ ]:

"""
Edge-to-Center Rectangular Subdivision
Given a division value div_val, and a data structure for faces 
(defined  by a properly structured three-dimensional collection of 
Segments),  a recursive subdivision routine is performed to a given 
number of generations.
"""
def subdivide():
    faces, subfaces = fgens[-1], []
    for fac in faces:
        #{b} a Point at the center of the face
        cen = Point.centroid([seg.ept for seg in fac])
        # midpoints of each edge
        pts = [seg.eval(div_val) for seg in fac]
        # starting edge sub-Segments
        ssubs = [Segment(seg.spt,pt) for seg,pt in zip(fac,pts)]
        # ending edge sub-Segments
        esubs = [Segment(pt,seg.ept) for seg,pt in zip(fac,pts)]
        # middle sub-Segments
        msubs = [a(pt,cen) for pt in pts]
        
        # weave together subdivided segs into four new faces
        subfaces.append( (ssubs[0],msubs[0],msubs[3].inverted(),esubs[3]) )
        subfaces.append( (ssubs[1],msubs[1],msubs[0].inverted(),esubs[0]) )
        subfaces.append( (ssubs[2],msubs[2],msubs[1].inverted(),esubs[1]) )
        subfaces.append( (ssubs[3],msubs[3],msubs[2].inverted(),esubs[2]) )
        
    fgens.append(subfaces)
    
# {a} subdivide for a given number of generations
for n in range(gens): 
    subdivide()


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.P02.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D07.jpg" style="width:400px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D08.jpg" style="width:400px; display: inline;">

## Recovering Values from Functions

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.P04.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D09.jpg" style="width:600px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D10.jpg" style="width:600px; display: inline;">

In [ ]:

"""
Corner-to-Center Triangular Subdivision
Each face in a given collection of faces is iteratively removed from 
the collection and subdivided, the results of which are collected in 
the subfaces List. At the end of each cycle, the subfaces List 
overwrites the faces List, and the process continues.
"""
def subdivide():
    subfaces = []
    # so long as faces remain,
    while faces:
        # remove the first remaining face in faces
        fac = faces.pop()
        cen = Point.centroid([seg.ept for seg in fac])
        # sub-Segments from each corner to the center
        csubs = [Segment(seg.ept,cen) for seg in fac]
        
        # {b} weave together subdivided Segments into three new faces
        subfaces.append( (fac[0],csubs[0],csubs[2].inverted())  )
        subfaces.append( (fac[1],csubs[1],csubs[0].inverted())  )
        subfaces.append( (fac[2],csubs[2],csubs[1].inverted())  )
        
    return subfaces

for n in range(gens): 
    # {a} overwrite faces with the results of the subdivision
    faces = subdivide()


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.P05.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Edge-to-Edge Triangular Subdivision
"""
def subdivide():
    subfaces = []
    while faces:
        fac = faces.pop()
        # midpoints of each edge
        pts = [seg.eval(0.5) for seg in fac]
        ssubs = [Segment(seg.spt,pt) for seg,pt in zip(fac,pts)]
        esubs = [Segment(pt,seg.ept) for seg,pt in zip(fac,pts)]
        msubs = [Segment(pa,pb) for pa,pb in zip(pts,pts[-1:]+pts[:-1])]
        
        subfaces.append( tuple([sub.inverted() for sub in msubs]) )
        subfaces.append( (ssubs[0],msubs[0],esubs[2]) )
        subfaces.append( (ssubs[1],msubs[1],esubs[0]) )
        subfaces.append( (ssubs[2],msubs[2],esubs[1]) )
        
    return subfaces

for n in range(gens): faces = subdivide()


## Passing Values to Functions

### Passing Values via Keyword

In [ ]:
"""
Passing Values via Keyword
Functions may be called using a positional assignment of arguments, 
or by keyword. The last two lines of this code are equivalent calls 
to the cs_eval_cyl function.
"""    
def cs_eval_cyl(cs,rad,ang):
    return cs.eval( Point( rad*math.cos(ang), rad*math.sin(ang) ) )

# positional argument assignment
cs_eval_cyl( CS(), 1.0, math.pi )
# keyword argument assignment
cs_eval_cyl( CS(), ang = math.pi, rad = 1.0)

### Default Values

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.C01.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Vector Interpolation
Calculates and returns the vector that results from an interpolation 
between two given vectors at parameter t.
"""
@staticmethod
def interpolate( va, vb, t=0.5 ):
    x = (1-t) * va.x + t * vb.x
    y = (1-t) * va.y + t * vb.y
    z = (1-t) * va.z + t * vb.z
    return Vec(x,y,z)

In [1]:
"""
CS on XY Plane
Returns a coordinate system on the world xy plane at a specified 
coordinate location. Optionally, one may define a vector that 
controls the orientation of the x_axis of the resulting CS on the 
Plane. The z coordinate of this vector is ignored. If this vector 
is not given, the world x-vector is used.
"""
@staticmethod
def on_xy( x, y, vec=None ):
    # default orientation vector is the unit-x vector
    if vec is None: vec = UX
    # ignore the z-coordinate of the given Vec
    vec.z = 0
    return CS( Point(x,y,0), vec, vec.cross(UZ.inverted()) )
    
# a CS at (1,1) at a 45deg angle
cs = CS.on_xy(1,1,vec=Vec(1,1))  

NameError: name 'CS' is not defined

### Packing and Unpacking Arguments

In [ ]:
"""
Threading Points with Positional Argument Packing
A function may be configured to receive any number of arguments using 
packing. Here, any number of given Points are threaded together to 
form a closed loop of Segments. Similar functionality could be 
achieved by defining a function that takes a single List of Points 
as an argument.
"""
def thread_pts(*pts):
    segs = [ Segment(pa,pb) for pa,pb in zip(pts[:-1],pts[1:]) ]
    return segs + [Segment(pts[-1],pts[0])]

segs = thread_pts(pa,pb,pc,pd,pe)

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D11.jpg" style="width:600px; display: inline;">

In [ ]:

"""
Plotting Points with Positional Argument Unpacking
Given values already stored in a collection, we may provide a regular 
function with a series of positional arguments by using the asterisk 
operator. Used in this way, this operator is sometimes called the 
"splat" operator. 
"""
crd = (0,0)
pt = Point(*crd)

crds = [(1,1),(2,2),(3,3,1)]
pts = [Point(*crd) for crd in crds]


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.05.D12.jpg" style="width:600px; display: inline;">

In [4]:
"""
Sorting Points by Polar Angle from First
Applying both argument packing and variable unpacking, all but the 
first of a given List of Points is sorted by its polar angle relative 
to the first Point in the List. A tuple of the first Point and a 
sorted List of the remaining Points is returned.
"""
def sort_by_angle(origin,*args):
    spts = sorted(args,key=lambda pt: Vec.ux().angle(Vec(origin,pt)))
    return origin, spts

opt, pts = sort_by_angle(*pts)

In [10]:
"""
CS on XY Plane using Keyword Argument Packing
Returns a coordinate system on the world xy plane at a specified 
coordinate location. One may define a vector that controls the 
orientation of the x_axis, or define the rotation from the world 
x-axis of the resulting CS on the Plane.
"""

def on_xy(x,y,**kargs):
    # define the orientation vector by whatever means
    if 'vec' in kargs: vec = kargs['vec']
    elif 'rot' in kargs: vec = Vec(math.cos( kargs['rot']),math.sin( kargs['rot'])) 
    else: vec = Vec.ux()
    
    # ignore the z-coordinate of the given Vec
    vec.z = 0
    return CS( Point(x,y,0), vec, vec.cross(UZ.inverted()) )
    
# a CS at (1,1) at a 60deg angle
cs = on_xy(1,1,rot=math.pi/3)
# a CS at (1,2) at a 45 deg angle
cs = on_xy(1,2,vec=Vec(1,1)) 


In [11]:

"""
Cylindrical Points using Keyword Argument Unpacking
Produces Points by cylindrical coordinates. A "table" of values is 
constructed as a collection of Dicts, each containing the required 
arguments for calling the cs_eval_cyl function. These are then 
iteratively passed to the function using keyword argument unpacking.
"""
def cs_eval_cyl(cs,rad,ang):
    return cs.eval( Point( rad*math.cos(ang), rad*math.sin(ang) ) )

pt_table = [
    { 'cs':CS(), 'rad':1.0, 'ang':0.0 },
    { 'cs':CS(), 'rad':2.0, 'ang':math.pi/2 },
    { 'cs':CS(), 'rad':3.0, 'ang':math.pi }
]

css = [cs_eval_cyl(**pt_dict) for pt_dict in pt_table]


### Flexible Argumentation

In [ ]:

"""
Vec Three Ways
The Decod.es library offers three ways to create a Vec: va is 
constructed by three coordinates, vb is constructed as the vector 
that spans between two given Points, and vc is constructed by 
copying the members of a given Vec or Point.
"""            
va = Vec(0,0,0)
vb = Vec(pa,pb)
vc = Vec(pc)

In [ ]:

"""
Vector Construction
To facilitate a flexible initialization of vectors, the Vec 
constructor method is designed to be able to receive three 
configuration of arguments. By assigning default values of None to 
each, and subsequently testing for the kind of argument that has 
been passed using a lambda function, we may reliably distinguish 
between these three configurations
"""
class Vec(Geometry):
    
    def __init__(self, a=None, b=None, c=None):
        # lambda to determine pointishness
        is_pt = lambda o: hasattr(o,'x') and hasattr(o,'y') and hasattr(o,'z')
        
        # we've been given three numbers, define a new Vec
        if a is not None and b is not None and c is not None :
            self.x, self.y, self.z = a ,b, c
        # we've been given two point-like things, create a Vec between
        elif is_pt(a) and is_pt(b):
            self.x, self.y, self.z = b.x - a.x, b.y - a.y, b.z - a.z
        # we've been given one point-like thing, copy it.
        elif is_pt(a):
            self.x, self.y, self.z = a.x, a.y, a.z

